In [4]:
# Load 20250425/hypsum.out

""" Date    Origin    Lat      Long      Depth    Mag No Gap Dmin  Rms  Erh  Erz  Erx      Cvxy      Cvxz      Cvyz     Oterr
250425 1733 16.60 40 49.33  28 26.25    5.0    0.0 13  86 20.6 0.25  2.8  7.0  2.3 0.131E+01 0.601E+01 0.457E+01 0.583E+00
"""

solved_latitude = 40 + 49.44 / 60
solved_longitude = 28 + 26.32 / 60
solved_depth = 5.0
solved_origin_time = "17:33:16.60"

# print the actual
evla=40.86
evlo=28.43
evdepth = 8.0
# Time of the picked event 2025-04-25T17:33:16
print(f"Actual Latitude: {evla}, Actual Longitude: {evlo}, Actual Depth: {evdepth} km, Time: 17.33.16")
print(f"Solved Latitude: {solved_latitude}, Solved Longitude: {solved_longitude}, Solved Depth: {solved_depth} km, Time: {solved_origin_time}")

Actual Latitude: 40.86, Actual Longitude: 28.43, Actual Depth: 8.0 km, Time: 17.33.16
Solved Latitude: 40.824, Solved Longitude: 28.438666666666666, Solved Depth: 5.0 km, Time: 17:33:16.60


In [5]:
# Load the used stations from 20250425/hyp.in
from obspy import read_inventory
import glob
import os

def load_stations(filename):
    """Load station names and pick times from hyp.in file"""
    stations = []
    with open(filename, 'r') as f:
        lines = f.readlines()
        # Skip header lines and process station picks
        for line in lines:
            if line.strip() and not line.startswith(' 2025') and 'Action:' not in line and 'STAT SP' not in line:
                parts = line.split()
                if len(parts) >= 1 and parts[0].isalpha():
                    station_name = parts[0]
                    phase = parts[1] if len(parts) > 1 else 'P'
                    if station_name not in [s[0] for s in stations]:  # Only add unique stations
                        stations.append((station_name, phase))
    return stations

def load_station_information(station_dir):
    """Load station locations from XML files"""
    station_info = {}
    station_files = glob.glob(f"{station_dir}/*.xml")
    
    for station_file in station_files:
        try:
            inv = read_inventory(station_file)
            for network in inv:
                for station in inv[0]:
                    station_code = station.code
                    station_info[station_code] = {
                        'latitude': station.latitude,
                        'longitude': station.longitude,
                        'network': network.code,
                        'full_code': f'{network.code}.{station.code}'
                    }
        except Exception as e:
            print(f"Error reading {station_file}: {e}")
    
    return station_info

# Load the stations used in the location
eqname = "20250425"
stations_used = load_stations(f"{eqname}/hyp.in")
station_info = load_station_information(f"{eqname}/stations")

print(f"Stations used in location: {len(stations_used)}")
for station, phase in stations_used:
    if station in station_info:
        print(f"  {station_info[station]['full_code']} - {phase} phase")
    else:
        print(f"  {station} - {phase} phase (no location info)")


Stations used in location: 10
  KO.GUZE - P phase
  KO.SILV - P phase
  KO.BOTS - P phase
  KO.BGKT - P phase
  KO.BRGA - P phase
  KO.KAVV - P phase
  KO.KCTX - P phase
  KO.CRLU - P phase
  KO.MRMT - P phase
  KO.GEML - P phase


In [6]:
# Load station residuals from print.out
def load_station_residuals(filename):
    """Load station residuals (time errors) from HYPOINVERSE print.out file"""
    residuals = {}
    with open(filename, 'r') as f:
        lines = f.readlines()
        # Find the section with station residuals
        in_residuals_section = False
        for line in lines:
            if 'stn   dist   azm  ain w phas' in line:
                in_residuals_section = True
                continue
            if in_residuals_section:
                if line.strip() == '' or 'Difference from previous' in line:
                    break
                parts = line.split()
                if len(parts) >= 13:
                    station_name = parts[0]
                    try:
                        residual = float(parts[12])  # res column (0-indexed: stn dist azm ain w phas ? calcphs hrmn tsec t-obs t-cal res)
                        if station_name not in residuals:
                            residuals[station_name] = []
                        residuals[station_name].append(residual)
                    except (ValueError, IndexError):
                        continue
    # Calculate average residual for stations with multiple phases
    avg_residuals = {}
    for station, res_list in residuals.items():
        avg_residuals[station] = sum([abs(r) for r in res_list]) / len(res_list)
    return avg_residuals

station_residuals = load_station_residuals(f"{eqname}/print.out")

# Create a map showing actual vs solved event locations and stations used
import folium

# Create a map centered between the actual and solved locations
center_lat = (evla + solved_latitude) / 2
center_lon = (evlo + solved_longitude) / 2
m = folium.Map(location=[center_lat, center_lon], zoom_start=10, tiles='OpenStreetMap')

# Add actual event location marker (red)
folium.Marker(
    location=[evla, evlo],
    popup=f'<b>Actual Event</b><br>Lat: {evla}<br>Lon: {evlo}<br>Depth: {evdepth} km<br>Time: 17:33:16',
    icon=folium.Icon(color='red', icon='star'),
    tooltip='Actual Event Location'
).add_to(m)

# Add solved event location marker (green)
folium.Marker(
    location=[solved_latitude, solved_longitude],
    popup=f'<b>Solved Event</b><br>Lat: {solved_latitude:.4f}<br>Lon: {solved_longitude:.4f}<br>Depth: {solved_depth} km<br>Time: {solved_origin_time}',
    icon=folium.Icon(color='green', icon='star'),
    tooltip='Solved Event Location (HYPOINVERSE)'
).add_to(m)

# Add line connecting actual and solved locations
folium.PolyLine(
    locations=[[evla, evlo], [solved_latitude, solved_longitude]],
    color='purple',
    weight=2,
    opacity=0.7,
    tooltip='Location Error'
).add_to(m)

# Function to get color based on residual (absolute value)
def get_color_for_residual(residual):
    """Return color based on residual: green (small) -> yellow -> red (large)"""
    if residual < 0.15:
        return '#00ff00'  # Green - excellent
    elif residual < 0.35:
        return '#ffff00'  # Yellow - good
    elif residual < 0.5:
        return '#ff7f00'  # Yellow - fair
    else:
        return '#ff0000'  # Red - poor


# Add stations used in the location solution with smaller circle markers colored by residual
for station_name, phase in stations_used:
    if station_name in station_info:
        info = station_info[station_name]
        residual = station_residuals.get(station_name, 0.0)
        color = get_color_for_residual(residual)
        
        folium.CircleMarker(
            location=[info['latitude'], info['longitude']],
            radius=6,  # Small circle
            popup=f"<b>{info['full_code']}</b><br>Lat: {info['latitude']:.4f}<br>Lon: {info['longitude']:.4f}<br>Avg Residual: {residual:.3f} sec",
            color='black',
            weight=1,
            fill=True,
            fillColor=color,
            fillOpacity=0.8,
            tooltip=f"{info['full_code']}: {residual:.3f}s"
        ).add_to(m)

# Add a legend for the color scale
legend_html = '''
<div style="position: fixed; 
            bottom: 50px; right: 50px; width: 180px; height: 150px; 
            background-color: white; border:2px solid grey; z-index:9999; 
            font-size:14px; padding: 10px">
<p style="margin: 0 0 5px 0;"><b>Residual (sec)</b></p>
<p style="margin: 2px 0;"><span style="background-color: #00ff00; padding: 3px 10px;">⬤</span> < 0.15 (Excellent)</p>
<p style="margin: 2px 0;"><span style="background-color: #ffff00; padding: 3px 10px;">⬤</span> 0.15-0.35 (Good)</p>
<p style="margin: 2px 0;"><span style="background-color: #ff7f00; padding: 3px 10px;">⬤</span> 0.35-0.50 (Fair)</p>
<p style="margin: 2px 0;"><span style="background-color: #ff0000; padding: 3px 10px;">⬤</span> ≥ 0.50 (Poor)</p>
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

# Calculate and display the location error
from obspy.geodetics import gps2dist_azimuth
distance_m, az1, az2 = gps2dist_azimuth(evla, evlo, solved_latitude, solved_longitude)
distance_km = distance_m / 1000

print(f"\nLocation Error: {distance_km:.2f} km")
print(f"Depth Error: {abs(evdepth - solved_depth):.2f} km")
print(f"\nStation Residuals (average absolute):")
for station, res in sorted(station_residuals.items(), key=lambda x: x[1]):
    print(f"  {station}: {res:.3f} sec")

# Display the map
m



Location Error: 4.06 km
Depth Error: 3.00 km

Station Residuals (average absolute):
  GEML: 0.070 sec
  BOTS: 0.100 sec
  SILV: 0.145 sec
  MRMT: 0.200 sec
  KCTX: 0.280 sec
  BRGA: 0.325 sec
  BGKT: 0.340 sec
  GUZE: 0.350 sec
  CRLU: 0.405 sec
  KAVV: 0.440 sec
